In [1]:
import pandas as pd
import numpy as np

# Read Excel file
file_path = 'NPS_Telegram.xlsx'
sheet_name = 'pre-processed'
df = pd.read_excel(file_path, sheet_name=sheet_name)

# Define categories
categories = ['Penyebaran Informasi', 'Tanggapan dan Response', 'Kepuasan Informasi dan Komunikasi']

# Create a dictionary to store results
nps_results = {}

# Calculate NPS for each category
for category in categories:
    # Classify responses
    df['Category'] = np.where(df[category] < 5, 'Detractor', np.where(df[category] >= 5, 'Promoter', 'Passive'))

    # Calculate NPS
    nps_df = df.groupby(['Batch', 'Category']).size().unstack(fill_value=0)
    nps_df['Total'] = nps_df.sum(axis=1)
    nps_df[f'NPS_{category}'] = (nps_df['Promoter'] / nps_df['Total']) - (nps_df['Detractor'] / nps_df['Total'])

    # Store results
    nps_results[category] = nps_df

# Create a new DataFrame for the results
nps_summary_df = pd.DataFrame()

# Update the new sheet with NPS results for each category
for category, result_df in nps_results.items():
    result_df.reset_index(inplace=True)
    result_df = result_df.rename(columns={f'NPS_{category}': f'{category}_NPS'})  # Rename NPS column for clarity
    if nps_summary_df.empty:
        nps_summary_df = result_df
    else:
        nps_summary_df = pd.merge(nps_summary_df, result_df[['Batch', f'{category}_NPS']], on='Batch', how='outer')

# Calculate the average NPS score
nps_columns = [f'{category}_NPS' for category in categories]
nps_summary_df['Average_NPS'] = nps_summary_df[nps_columns].mean(axis=1)

# Display the results
print(nps_summary_df)


Category      Batch  Detractor  Promoter  Total  Penyebaran Informasi_NPS  \
0        2023-08-01         21       124    145                  0.710345   
1        2023-09-01         15        95    110                  0.727273   
2        2023-10-01         11       135    146                  0.849315   
3        2023-11-01         32       180    212                  0.698113   
4        2023-12-01         33       204    237                  0.721519   
5        2024-01-01          9        86     95                  0.810526   

Category  Tanggapan dan Response_NPS  Kepuasan Informasi dan Komunikasi_NPS  \
0                           0.765517                               0.710345   
1                           0.800000                               0.654545   
2                           0.835616                               0.835616   
3                           0.811321                               0.773585   
4                           0.755274                             

In [2]:
print(df.columns)

Index(['Batch', 'Penyebaran Informasi', 'Tanggapan dan Response',
       'Kepuasan Informasi dan Komunikasi', 'Penyebaran Informasi.1',
       'Tanggapan dan Response.1', 'Kepuasan Informasi dan Komunikasi.1',
       'Category'],
      dtype='object')
